In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium import webdriver
from datetime import timedelta
from datetime import datetime
import win32api,win32con
from pathlib import Path
from time import sleep
import pyautogui
import os, glob
import shutil
import requests
import json

#desativa o capslook
print('DESATIVAR CAPSLOCK DO TECLADO, SE ESTIVER ATIVO')
if win32api.GetKeyState(win32con.VK_CAPITAL) == 1:
    pyautogui.hotkey('capslock')
    
#inicia o chrome ignorando o erro
print('INICIAR CHROME IGNORANDO CERTIFICADO INVALIDO')
options = webdriver.ChromeOptions()

options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-blink-features=AutomationControlled') # Pular Captcha
driver = webdriver.Chrome(options=options)
driver.get(url='https://10.19.254.115/reports/browse/Controly/Controle%20Cont%C3%A1bil')
# driver.minimize_window()

#login no portalBI
print('FAZER LOGIN PORTAL BI ATRAVÉS AUTOMAÇÃO GUI - NAVEGADOR NÃO PODE ESTAR MINIMIZADO')
user = 'Victor.Paiva'
password = 'r4Tsz^6_4+'

sleep(1)
pyautogui.write(user)
pyautogui.hotkey('tab')
pyautogui.write(password)
sleep(2)
pyautogui.press('enter')

#seleciona qual relatório quero - nesse caso o "Processamento Bandeira por Bin"
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[26]/report-tile/rs-tile/a[1]/span[2]/span/span')))
driver.find_element(By.XPATH,'//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[26]/report-tile/rs-tile/a[1]/span[2]/span/span').click()


print('ALTERNAR IFRAME')
#driver.switch_to.default_content()
driver.switch_to.frame(0)

#DEFINE DATAS
#define datas dos relatórios que usam 1 data apenas
def ultimo_dia_util(data):
    while data.weekday() >= 5:
        data -= timedelta(days=1)
    return data

hoje = datetime.today()
fim = ultimo_dia_util(hoje-timedelta(days=1))
if hoje.weekday() ==1:
    inicio = ultimo_dia_util(hoje-timedelta(days=3))
else:
    inicio = fim

inicio = inicio.strftime('%d/%m/%Y')
fim = fim.strftime('%d/%m/%Y')


#selecionando visão - "Pagamentos"
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl05_ddValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_ddValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_ddValue"]').send_keys("p")
sleep(2)

#Preenchendo a data:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]').send_keys(fim)
sleep(2)

#clica em exibir relatório
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl00"]')))
print("Clicando no botão")
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl00"]').click()
sleep(2)

#aguardando relatório
print('AGUARDAR RELATORIO SER GERADO')
while driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_AsyncWait_Wait"]/table/tbody/tr/td[2]/div/a').text == 'Cancelar':
    sleep(3)

#faz o dowload do primeiro arquivo - EXCEL
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[2]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(2) > a').click();")
sleep(15)

#faz o dowload do PRIMEIRO arquivo - CSV
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[7]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(7) > a').click();")
sleep(15)


#SEGUNDO RELATÓRIO - A data e o tipo já estão definidos, precisamos apenas de alterar a visão - Compras
#selecionando visão
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl05_ddValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_ddValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_ddValue"]').send_keys("c")
sleep(2)

#clica em exibir relatório
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl00"]')))
print("Clicando no botão")
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl00"]').click()
sleep(2)

#aguardando relatório
print('AGUARDAR RELATORIO SER GERADO')
while driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_AsyncWait_Wait"]/table/tbody/tr/td[2]/div/a').text == 'Cancelar':
    sleep(3)

#faz o dowload do segundo arquivo - EXCEL
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[2]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(2) > a').click();")
sleep(15)

#faz o dowload do QUINTO arquivo - CSV
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[7]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(7) > a').click();")
sleep(15)

#TERCEIRO RELATÓRIO - Valor parcelado por BIN

#options.add_argument('--ignore-ssl-errors=yes')
#options.add_argument('--ignore-certificate-errors')
#options.add_argument('--disable-blink-features=AutomationControlled') # Pular Captcha
#driver = webdriver.Chrome(options=options)
driver.get(url='https://10.19.254.115/reports/browse/Controly/Gerencial')

#seleciona qual relatório quero - Valor parcelado por BIN
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[11]/report-tile/rs-tile/a[1]/span[2]/span/span')))
driver.find_element(By.XPATH,'//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[11]/report-tile/rs-tile/a[1]/span[2]/span/span').click()
sleep(2)

print('ALTERNAR IFRAME')
#driver.switch_to.default_content()
driver.switch_to.frame(0)

#Preenchendo a data inicial:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]').send_keys(inicio)
sleep(2)

#Preenchendo a data final:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl05_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_txtValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_txtValue"]').send_keys(fim)
sleep(2)

#clica em exibir relatório
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl00"]')))
print("Clicando no botão")
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl00"]').click()
sleep(2)

#aguardando relatório
print('AGUARDAR RELATORIO SER GERADO')
while driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_AsyncWait_Wait"]/table/tbody/tr/td[2]/div/a').text == 'Cancelar':
    sleep(3)

#faz o dowload do terceiro arquivo EXCEL
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[2]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(2) > a').click();")
sleep(15)

#faz o dowload do QUINTO arquivo - CSV
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[7]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(7) > a').click();")
sleep(15)

#QUARTO RELATÓRIO - Transações Tecban - Conciliação
driver.get(url='https://10.19.254.115/reports/browse/Controly/Controle%20Cont%C3%A1bil')

#seleciona qual relatório quero - Transações Tecban - Conciliação
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[38]/report-tile/rs-tile/a[1]/span[2]/span/span')))
driver.find_element(By.XPATH,'//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[38]/report-tile/rs-tile/a[1]/span[2]/span/span').click()
sleep(2)

print('ALTERNAR IFRAME')
#driver.switch_to.default_content()
driver.switch_to.frame(0)

#Selecionar todos os BIN's
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl07_ddValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl07_ddValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl07_ddValue"]').send_keys("<")
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl07_ddValue"]').click()
sleep(2)

#Selecionar TC - Selecionar Tudo
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl13_txtValue"]').click()
sleep(2)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl13_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl13_txtValue"]').click()
sleep(2)
print('SELECIONAR TUDO')
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl13_divDropDown_ctl00"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl13_divDropDown_ctl00"]').click()
sleep(2)

#Preenchendo a data inicial:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl09_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl09_txtValue"]').click()
sleep(2)
element=driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl09_txtValue"]')
actions=ActionChains(driver)
actions.click(element).click(element).click(element).perform()#tres cliques
sleep(2)
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl09_txtValue"]').send_keys(inicio)
sleep(2)

#Preenchendo a data final:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl11_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl11_txtValue"]').click()
sleep(2)
element=driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl11_txtValue"]')
actions=ActionChains(driver)
actions.click(element).click(element).click(element).perform()#tres cliques
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl11_txtValue"]').send_keys(fim)
sleep(2)

#clica em exibir relatório
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl00"]')))
print("Clicando no botão")
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl00"]').click()
sleep(2)

#aguardando relatório
print('AGUARDAR RELATORIO SER GERADO')
while driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_AsyncWait_Wait"]/table/tbody/tr/td[2]/div/a').text == 'Cancelar':
    sleep(3)

#faz o dowload do terceiro arquivo EXCEL
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[2]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(2) > a').click();")
sleep(15)

#faz o dowload do QUINTO arquivo - CSV
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[7]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(7) > a').click();")
sleep(15)


#QUINTO RELATÓRIO - Conciliação Contábil - GRANDE SAI APENAS EM CSV
driver.get(url='https://10.19.254.115/reports/browse/Controly/Controle%20Cont%C3%A1bil')

#seleciona qual relatório quero - Conciliação Contábil
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[12]/report-tile/rs-tile/a[1]/span[2]/span/span')))
driver.find_element(By.XPATH,'//*[@id="main"]/div/div/section[2]/tiles-view/section/div/div/div/ul/li[12]/report-tile/rs-tile/a[1]/span[2]/span/span').click()
sleep(2)

print('ALTERNAR IFRAME')
#driver.switch_to.default_content()
driver.switch_to.frame(0)

#Preenchendo a data inicial:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl03_txtValue"]').send_keys(inicio)
sleep(2)

#Preenchendo a data final:
print('Prenchendo Data', inicio)
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl05_txtValue"]')))
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_txtValue"]').click()
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl05_txtValue"]').send_keys(fim)
sleep(2)

#clica em exibir relatório
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl04_ctl00"]')))
print("Clicando no botão")
driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_ctl04_ctl00"]').click()
sleep(2)

#aguardando relatório
print('AGUARDAR RELATORIO SER GERADO')
while driver.find_element(By.XPATH,'//*[@id="ReportViewerControl_AsyncWait_Wait"]/table/tbody/tr/td[2]/div/a').text == 'Cancelar':
    sleep(3)

#faz o dowload do QUINTO arquivo - CSV
print('FAZER DOWNLOAD DO ARQUIVO')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ReportViewerControl_ctl05_ctl04_ctl00_Menu"]/div[7]/a')))
driver.execute_script("document.querySelector('#ReportViewerControl_ctl05_ctl04_ctl00_Menu > div:nth-child(7) > a').click();")
sleep(15)

#copia os arquivos da pasta dowloads para a pasta no drive
pasta_origem = r"C:\Users\u004183\Downloads"
pasta_destino = r"G:\Drives compartilhados\arquitetura\Documentos\Arquitetura Operacional\BACKOFFICE FINANCEIRO\Cartões\Arquivos_Portal_BI"

for arquivo in os.listdir(pasta_origem):
  caminho_origem = os.path.join(pasta_origem,arquivo)
  caminho_destino = os.path.join(pasta_destino,arquivo)

  if os.path.isfile(caminho_origem):
    shutil.copy2(caminho_origem,caminho_destino)
    print(f'Arquivo{arquivo} copiado com sucesso para pasta{pasta_destino}')
print('Cópia completa!')

#avisa no slack
url = "https://hooks.slack.com/triggers/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"
message = {
  'text': "Arquivos do Portal BI baixados e copiados para o drive."
}

response = requests.post(
  url, data=json.dumps(message),
  headers={'Content-Type': 'aplication/json'}
)